In [2]:
from ultralytics import YOLO

# Download and load the model
model = YOLO('yolov8n.pt')  # This will automatically download the model if not present

# Method 2: Direct download and load
import torch

# Download using Python's urllib
import urllib.request
import os

def download_yolo_model():
    model_url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt"
    save_path = "yolov8n.pt"
    
    if not os.path.exists(save_path):
        print(f"Downloading YOLOv8 model to {save_path}...")
        urllib.request.urlretrieve(model_url, save_path)
        print("Download completed!")
    else:
        print("Model already exists!")
    
    return save_path

# Run the download function
model_path = download_yolo_model()

# Test the model
def test_yolo_model():
    try:
        model = YOLO(model_path)
        print("Model loaded successfully!")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

# Test the installation
if test_yolo_model():
    print("YOLOv8 is ready to use!")

100%|██████████| 6.25M/6.25M [00:09<00:00, 677kB/s]


Model already exists!
Model loaded successfully!
YOLOv8 is ready to use!


by this code we can check vehcle is move or not(EXTERNAL VIDEO)

In [16]:
import cv2
import numpy as np
from ultralytics import YOLO
import time
import os

class LightVehicleDetector:
    def __init__(self, video_path):
        # Check if video file exists
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
            
        # Initialize video capture
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise ValueError(f"Error opening video file: {video_path}")
            
        # Get video properties
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"Video Properties:")
        print(f"Resolution: {self.frame_width}x{self.frame_height}")
        print(f"FPS: {self.fps}")
        print(f"Total Frames: {self.total_frames}")
        
        # Calculate target size maintaining aspect ratio
        target_width = 640
        self.target_size = (target_width, int(target_width * self.frame_height / self.frame_width))
        
        # Load YOLO model
        try:
            self.yolo = YOLO('yolov8n.pt')  # Using nano model
            self.yolo.conf = 0.5  # Higher confidence threshold
            print("YOLO model loaded successfully!")
        except Exception as e:
            print(f"Error loading YOLO model: {e}")
            raise
        
        # Track previous frame for optical flow calculation
        self.prev_gray = None
        self.process_every_n_frames = 3
        self.frame_count = 0
        
    def detect_vehicles(self, frame):
        """Simplified vehicle detection"""
        results = self.yolo.predict(frame, classes=[2, 3, 5, 7], verbose=False)  # car, motorcycle, bus, truck
        return results[0].boxes
    
    def estimate_optical_flow(self, prev_gray, gray):
        """Estimate optical flow between two frames"""
        # Calculate dense optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        # Compute magnitude and angle of flow
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        return mag
    
    def is_vehicle_moving(self, mag, box):
        """Check if the vehicle inside the bounding box is moving"""
        # Extract the region of interest (ROI) where the vehicle is located
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        roi_mag = mag[y1:y2, x1:x2]
        
        # Calculate average motion magnitude in the ROI
        avg_magnitude = np.mean(roi_mag)
        
        # Set a threshold for detecting motion (tune this value)
        moving_threshold = 1.5
        
        # Return True if the vehicle is moving, otherwise False
        return avg_magnitude > moving_threshold
    
    def run(self):
        print("\nControls:")
        print("'q' - Quit")
        print("'p' - Pause/Resume")
        print("'>' - Speed up")
        print("<' - Slow down")
        
        paused = False
        playback_speed = 1.0
        
        while True:
            if not paused:
                ret, frame = self.cap.read()
                if not ret:
                    print("End of video reached")
                    break
                
                # Skip frames based on process_every_n_frames
                self.frame_count += 1
                if self.frame_count % self.process_every_n_frames != 0:
                    continue
                
                # Resize frame for faster processing
                frame_resized = cv2.resize(frame, self.target_size)
                
                # Convert to grayscale for optical flow calculation
                gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
                
                if self.prev_gray is not None:
                    # Detect vehicles
                    boxes = self.detect_vehicles(frame_resized)
                    
                    # Estimate optical flow
                    mag = self.estimate_optical_flow(self.prev_gray, gray)
                    
                    # Process each detection
                    for box in boxes:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        confidence = float(box.conf[0])
                        class_id = int(box.cls[0])
                        
                        # Get class name
                        class_names = ['car', 'motorcycle', 'bus', 'truck']
                        class_name = class_names[class_id-2] if class_id-2 < len(class_names) else 'vehicle'
                        
                        # Check if the vehicle is moving
                        is_moving = self.is_vehicle_moving(mag, box)
                        
                        # Draw bounding box
                        color = (0, 255, 0) if is_moving else (0, 0, 255)
                        cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 2)
                        
                        # Draw label with confidence
                        label = f"{class_name} {'Moving' if is_moving else 'Stopped'} {confidence:.2f}"
                        cv2.putText(frame_resized, label, (x1, y1-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    
                # Update the previous frame for optical flow calculation
                self.prev_gray = gray
                
                # Show frame
                cv2.imshow('Vehicle Detection (Optical Flow)', frame_resized)
            
            # Handle keyboard input
            key = cv2.waitKey(int(1000/(self.fps*playback_speed))) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('p'):
                paused = not paused
                print("Video Paused" if paused else "Video Resumed")
            elif key == ord('.'):  # Speed up
                playback_speed = min(4.0, playback_speed + 0.25)
                print(f"Playback Speed: {playback_speed}x")
            elif key == ord(','):  # Slow down
                playback_speed = max(0.25, playback_speed - 0.25)
                print(f"Playback Speed: {playback_speed}x")
        
        self.cap.release()
        cv2.destroyAllWindows()

# Run the detector
if __name__ == "__main__":
    try:
        # Replace with your video file path
        video_path = "C:/Users/Lahiru/OneDrive - University of Jaffna/Desktop/Research/Optical flow/4.mp4"
        detector = LightVehicleDetector(video_path)
        detector.run()
    except Exception as e:
        print(f"Error: {e}")


Video Properties:
Resolution: 1280x720
FPS: 25
Total Frames: 180
YOLO model loaded successfully!

Controls:
'q' - Quit
'p' - Pause/Resume
'>' - Speed up
<' - Slow down
End of video reached


by this code we can check vehcle is move or not(EXTERNAL CAMERA)

In [19]:
import cv2
import numpy as np
from ultralytics import YOLO
import time

class LightVehicleDetector:
    def __init__(self, video_source=0):
        # Initialize video capture (0 for the default webcam)
        self.cap = cv2.VideoCapture(video_source)
        if not self.cap.isOpened():
            raise ValueError(f"Error opening video source: {video_source}")
            
        # Get video properties
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS)) if video_source != 0 else 30  # Default FPS for webcam
        self.total_frames = float('inf')  # For a live webcam feed, there are infinite frames
        
        print(f"Video Properties:")
        print(f"Resolution: {self.frame_width}x{self.frame_height}")
        print(f"FPS: {self.fps}")
        
        # Calculate target size maintaining aspect ratio
        target_width = 640
        self.target_size = (target_width, int(target_width * self.frame_height / self.frame_width))
        
        # Load YOLO model
        try:
            self.yolo = YOLO('yolov8n.pt')  # Using nano model
            self.yolo.conf = 0.5  # Higher confidence threshold
            print("YOLO model loaded successfully!")
        except Exception as e:
            print(f"Error loading YOLO model: {e}")
            raise
        
        # Track previous frame for optical flow calculation
        self.prev_gray = None
        self.process_every_n_frames = 3
        self.frame_count = 0
        
    def detect_vehicles(self, frame):
        """Simplified vehicle detection"""
        results = self.yolo.predict(frame, classes=[2, 3, 5, 7], verbose=False)  # car, motorcycle, bus, truck
        return results[0].boxes
    
    def estimate_optical_flow(self, prev_gray, gray):
        """Estimate optical flow between two frames"""
        # Calculate dense optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        # Compute magnitude and angle of flow
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        return mag
    
    def is_vehicle_moving(self, mag, box):
        """Check if the vehicle inside the bounding box is moving"""
        # Extract the region of interest (ROI) where the vehicle is located
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        roi_mag = mag[y1:y2, x1:x2]
        
        # Calculate average motion magnitude in the ROI
        avg_magnitude = np.mean(roi_mag)
        
        # Set a threshold for detecting motion (tune this value)
        moving_threshold = 1.5
        
        # Return True if the vehicle is moving, otherwise False
        return avg_magnitude > moving_threshold
    
    def run(self):
        print("\nControls:")
        print("'q' - Quit")
        print("'p' - Pause/Resume")
        print("'>' - Speed up")
        print("<' - Slow down")
        
        paused = False
        playback_speed = 1.0
        
        while True:
            if not paused:
                ret, frame = self.cap.read()
                if not ret:
                    print("Failed to capture frame")
                    break
                
                # Skip frames based on process_every_n_frames
                self.frame_count += 1
                if self.frame_count % self.process_every_n_frames != 0:
                    continue
                
                # Resize frame for faster processing
                frame_resized = cv2.resize(frame, self.target_size)
                
                # Convert to grayscale for optical flow calculation
                gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
                
                if self.prev_gray is not None:
                    # Detect vehicles
                    boxes = self.detect_vehicles(frame_resized)
                    
                    # Estimate optical flow
                    mag = self.estimate_optical_flow(self.prev_gray, gray)
                    
                    # Process each detection
                    for box in boxes:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        confidence = float(box.conf[0])
                        class_id = int(box.cls[0])
                        
                        # Get class name
                        class_names = ['car', 'motorcycle', 'bus', 'truck']
                        class_name = class_names[class_id-2] if class_id-2 < len(class_names) else 'vehicle'
                        
                        # Check if the vehicle is moving
                        is_moving = self.is_vehicle_moving(mag, box)
                        
                        # Draw bounding box
                        color = (0, 255, 0) if is_moving else (0, 0, 255)
                        cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 2)
                        
                        # Draw label with confidence
                        label = f"{class_name} {'Moving' if is_moving else 'Stopped'} {confidence:.2f}"
                        cv2.putText(frame_resized, label, (x1, y1-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    
                # Update the previous frame for optical flow calculation
                self.prev_gray = gray
                
                # Show frame
                cv2.imshow('Vehicle Detection (Optical Flow)', frame_resized)
            
            # Handle keyboard input
            key = cv2.waitKey(int(1000/(self.fps*playback_speed))) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('p'):
                paused = not paused
                print("Video Paused" if paused else "Video Resumed")
            elif key == ord('.'):  # Speed up
                playback_speed = min(4.0, playback_speed + 0.25)
                print(f"Playback Speed: {playback_speed}x")
            elif key == ord(','):  # Slow down
                playback_speed = max(0.25, playback_speed - 0.25)
                print(f"Playback Speed: {playback_speed}x")
        
        self.cap.release()
        cv2.destroyAllWindows()

# Run the detector
if __name__ == "__main__":
    try:
        # Using webcam (default camera index 0)
        detector = LightVehicleDetector(0)
        detector.run()
    except Exception as e:
        print(f"Error: {e}")


Video Properties:
Resolution: 640x480
FPS: 30
YOLO model loaded successfully!

Controls:
'q' - Quit
'p' - Pause/Resume
'>' - Speed up
<' - Slow down


i have to imrove the code